In [5]:
import matplotlib
from emoatlas import EmoScores

import nltk
# nltk.download('wordnet')
import networkx as nx
import numpy as np
import pandas as pd
import shapely
import os
import matplotlib.pyplot as plt
import itertools
import plotly.express as px
from tqdm import tqdm

src_path = os.getcwd()[0:-4]

text_font = 20

In [6]:
emos = EmoScores()

In [7]:
df = pd.read_csv(src_path+'data/articles_final.csv')
idx = np.where(df['language'] == 'English')

df = df.loc[idx]
df.head()

,url,domain,title,bias_rating,language,sourcecountry,date,russia,ukraine,nato
0,http://www.9news.com/news/politics/russia-engi...,9news.com,Russia engineered election hacks and meddling ...,LEAST BIASED,English,United States,2017-01-09 13:15:00,1,0,0
1,http://www.9news.com/news/military/fort-carson...,9news.com,Fort Carson involved in Operation Atlantic Re...,LEAST BIASED,English,United States,2017-01-10 01:45:00,0,0,0
2,http://www.9news.com/news/nation-now/russian-a...,9news.com,Russian ambassador Sergey Kislyak leaves trail...,LEAST BIASED,English,United States,2017-03-03 00:15:00,1,0,0
3,http://www.9news.com/news/nation-now/america-c...,9news.com,America cant be first without Europe,LEAST BIASED,English,United States,2017-03-24 19:00:00,0,0,0
4,http://www.9news.com/article/news/nation-now/r...,9news.com,9news . com,LEAST BIASED,English,United States,2018-01-10 18:45:00,0,0,0


In [12]:
bias_list = ['LEFT', 'RIGHT']
keywords = ['russia', 'nato']
emotion_list = ['anger', 'disgust',  'sadness', 'fear', 'anticipation', 'trust', 'surprise', 'joy']

for bias in bias_list:

    temp_df = df[df['bias_rating'].str.contains(bias)]

    for a, b in itertools.combinations(keywords, 2):
        idx = np.where((temp_df[a] == 1) & (temp_df[b] == 1))[0]
        comention_df = temp_df.iloc[idx]

        a_emotion_list = [0]*len(emotion_list)
        b_emotion_list = [0]*len(emotion_list)

        for i in tqdm(range(len(comention_df))):
            sample_text = comention_df.iloc[i]['title']
            fmnt = emos.formamentis_network(sample_text) 

            bag_of_words_a = " ".join([pair[1] for pair in fmnt.edges if pair[0].lower() == a])
            bag_of_words_b = " ".join([pair[1] for pair in fmnt.edges if pair[0].lower() == b])
            
            zscores_a = emos.zscores(bag_of_words_a)
            zscores_b = emos.zscores(bag_of_words_b)

            for j, emotion in enumerate(emotion_list):
                a_emotion_list[j] += (zscores_a[emotion]/len(comention_df))
                b_emotion_list[j] += (zscores_b[emotion]/len(comention_df))


        df_a = pd.DataFrame(dict(r=a_emotion_list, theta=emotion_list))
        fig = px.line_polar(df_a, r='r', theta='theta', line_close=True)
        font=dict(
        family="Courier New, monospace",
        size=30,  # Set the font size here
        color="RebeccaPurple"
    )
        fig.write_image(src_path +f'/figures/{bias}_{a}.png')
        fig.show()


        df_b = pd.DataFrame(dict(r=b_emotion_list, theta=emotion_list))
        fig = px.line_polar(df_b, r='r', theta='theta', line_close=True)
        font=dict(
        family="Courier New, monospace",
        size=30,  # Set the font size here
        color="RebeccaPurple"
    )
        fig.write_image(src_path +f'/figures/{bias}_{b}.png')
        fig.show()

100%|██████████| 2871/2871 [00:56<00:00, 50.60it/s]


100%|██████████| 1418/1418 [00:28<00:00, 50.16it/s]
